In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from keras.callbacks import LearningRateScheduler
from tqdm import tqdm
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, Activation, BatchNormalization, Conv2D, MaxPooling2D,AveragePooling2D, Conv2DTranspose, Reshape, UpSampling2D
from keras.models import Sequential, Model, load_model
from keras.regularizers import l2
import tensorflow as tf

In [3]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [4]:
#encoder = load_model('S2FA1vg(1000개 샘플).h5')

In [5]:
#encoder.summary()

In [6]:
X = np.load('Picture_feature_4096.npy')
Y = np.load('power_law_comp.npy')
Y_predict = np.load('Decoder_X_4096.npy')

In [21]:
A = Y_predict[0:10,]
B = Y[0:10,]
C = np.pad(B,((0,0),(0,1),(0,4),(0,0)))


In [8]:
#Y_predict = encoder.predict(Y)
#print(Y_predict)

In [9]:
#np.save("Decoder_X", Y_predict)

In [22]:
print(np.shape(Y_predict))
from tensorflow.keras.optimizers import Adam
from keras import losses
adam = Adam(learning_rate=0.001, beta_1=0.5, beta_2=0.9799, amsgrad=False)

(3753, 4096)


In [23]:
decoder = Sequential()
#decoder.add(Dense(2622, activation="softmax", input_shape = (Y_predict.shape[1],2622)))
#decoder.add(Dense(2622, activation="softmax", input_shape = (2622,)))
#decoder.add(Dense(2622, activation="softmax"))
#decoder.add(Dense(4096, activation="relu"))
decoder.add(Dense(4096, activation="relu", input_shape = (4096, )))
decoder.add(Dense(25440, activation="relu"))
#decoder.add(Flatten())
decoder.add(Reshape((1, 48, 530)))
decoder.add(BatchNormalization())
decoder.add(UpSampling2D((1,3)))

#decoder.add(BatchNormalization())
#decoder.add(AveragePooling2D(pool_size=(1,1), strides=3, padding='valid'))
#decoder.add(BatchNormalization())
decoder.add(Conv2DTranspose(530, kernel_size = 4, strides=2, activation='relu'))#, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
decoder.add(BatchNormalization())

decoder.add(Conv2DTranspose(530, kernel_size = 4,  strides=2,activation='relu'))#, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
decoder.add(BatchNormalization())
decoder.add(Conv2DTranspose(530, kernel_size = 4, strides=1, activation='relu'))#, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
decoder.add(UpSampling2D((2,1)))
decoder.add(BatchNormalization())
decoder.add(Conv2DTranspose(275, kernel_size = 4, strides=1, activation='relu'))#, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
decoder.add(UpSampling2D((2,1)))
decoder.add(BatchNormalization())
decoder.add(Conv2DTranspose(140, kernel_size = 3, strides=1, activation='relu'))#, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
decoder.add(UpSampling2D((2,1)))
decoder.add(BatchNormalization())
decoder.add(Conv2DTranspose(140, kernel_size = 4, strides=1, activation='relu'))#, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
decoder.add(UpSampling2D((2,1)))
decoder.add(BatchNormalization())
decoder.add(Conv2DTranspose(140, kernel_size = 4, strides=1, activation='relu'))#, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
decoder.add(BatchNormalization())
decoder.add(Conv2DTranspose(70,  kernel_size = 4, strides=1, activation='relu'))#, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
decoder.add(BatchNormalization())


decoder.add(Conv2DTranspose(70,  kernel_size = 4, strides=1, activation='relu' ))
decoder.add(BatchNormalization())
decoder.add(Conv2DTranspose(2,  kernel_size = 4, strides=1, activation='relu'))#, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
#decoder.summary()
decoder.compile(optimizer=adam, loss="mean_absolute_error", metrics=["mse"])

In [25]:
decoder.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_7 (Dense)              (None, 25440)             104227680 
_________________________________________________________________
reshape_3 (Reshape)          (None, 1, 48, 530)        0         
_________________________________________________________________
batch_normalization_30 (Batc (None, 1, 48, 530)        2120      
_________________________________________________________________
up_sampling2d_15 (UpSampling (None, 1, 144, 530)       0         
_________________________________________________________________
conv2d_transpose_30 (Conv2DT (None, 4, 290, 530)       4494930   
_________________________________________________________________
batch_normalization_31 (Batc (None, 4, 290, 530)      

In [ ]:
# DECREASE LEARNING RATE EACH EPOCH
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
print(annealer)
# TRAIN NETWORKS
epochs = 1
decoder.fit(A,C, batch_size=1,
      epochs = epochs ,  
      validation_split=0.1, callbacks=[annealer], verbose=1)

9/9 [==============================] - 14s 1s/step - loss: 0.6096 - mse: 0.9102 - val_loss: 0.5070 - val_mse: 0.4873


In [ ]:
decoder.save("/content/drive/My Drive/Decoder_10ea.h5")